# Airbnb Prices in Europe Project: ETL Part

> The Airbnb project is my personal quest towards getting better in my general data skills. This notebook is a part of this journey. Its goal is to take the original kaggle datasets that can be found [here](https://www.kaggle.com/datasets/thedevastator/airbnb-prices-in-european-cities) and merge them into one big and clean data frame where I can use it as the main source for my next and more analytical notebooks. More details on the project and about me can be found [here](https://github.com/rocha-felipeferreira) and [here](https://www.kaggle.com/rochafelipeferreira).

## EXTRACT
By looking carefully at the [webpage](https://www.kaggle.com/datasets/thedevastator/airbnb-prices-in-european-cities) where we have found all the files that are going to be used as my main data sources, I have noticed that they are broken into different parts. They are basically divided into countries and the data for the weekday and the information for the weekend. For example, Amsterdam has two CSV files: `amsterdam_weekends.csv` and `amsterdam_weekdays.csv`. 

I checked before and made sure every dataset has the same column names. Having this in mind, I decided to merge all datasets into one big file that can be used for my future projects. Thus, in this part, I create a function to implement this, i.e., to extract all the information needed into one file.


### Creating one single dataset

In [37]:
# Libraries
import numpy as np
import pandas as pd
import os
pd.set_option('display.max_columns', None)

try: 
    from dtype_diet import report_on_dataframe, optimize_dtypes
except:
    !pip install dtype_diet
    from dtype_diet import report_on_dataframe, optimize_dtypes

In [38]:
# Important variables to be used in my custom function to collapse all tables into one
filename_for_create_one_dataframe = []

for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        full_file_name = os.path.join(dirname, filename)
        filename_splitted = filename.split('_')
        filename_for_create_one_dataframe.append(filename_splitted)

In [39]:
# Function to create the dataset
def create_one_dataframe(location, period):
    if period == 'weekdays':
        raw_df_weekdays = pd.read_csv(f'/kaggle/input/airbnb-price-determinants-in-europe/{location}_{period}.csv')
        raw_df_weekdays['is_weekday'] = True
        raw_df_weekdays['location'] = location
        return raw_df_weekdays
    else:
        raw_df_weekends = pd.read_csv(f'/kaggle/input/airbnb-price-determinants-in-europe/{location}_{period}.csv')
        raw_df_weekends['is_weekday'] = False
        raw_df_weekends['location'] = location
        return raw_df_weekends

In [40]:
# Applying the previous function to all files and uniting all datasets
one_df = []

for arquivo in filename_for_create_one_dataframe:
    location = arquivo[0]
    period = arquivo[1].replace('.csv', '')
    df = create_one_dataframe(location, period)
    one_df.append(df)

one_dataframe = pd.concat(one_df)

## TRANSFORM
Optimisation is key when dealing with small or big datasets. After all, who doesn't want to run an analysis in the fastest time possible? In this sense, I decided to use some tools to try and improve the size of the big final dataset. Here, I adopted the following steps: 
- I explored a bit of the dataset
- I dropped some unnecessary columns
- I manually transformed some string columns from the object to the category data type
- Finally, I also used the functions `report_on_dataframe` and `optimize_dtypes` from the dtype_diet package to check whether I could use a better data type for my columns and, if that was the case, to implement the changes.

### Cleaning and Optimising the Dataset

In [41]:
# Exploring the Dataset
one_dataframe.sample(10)

,Unnamed: 0,realSum,room_type,room_shared,room_private,person_capacity,host_is_superhost,multi,biz,cleanliness_rating,guest_satisfaction_overall,bedrooms,dist,metro_dist,attr_index,attr_index_norm,rest_index,rest_index_norm,lng,lat,is_weekday,location
1765,1765,412.286272,Entire home/apt,False,False,4.0,False,0,1,10.0,100.0,1,9.320768,0.297278,143.380735,9.966554,286.947359,5.138032,0.00663,51.51154,True,london
748,748,145.185047,Private room,False,True,2.0,False,0,1,9.0,86.0,1,0.527409,0.243494,650.175392,14.408015,1614.695462,35.156466,12.49600,41.90000,True,rome
1821,1821,179.738945,Entire home/apt,False,False,6.0,False,1,0,10.0,100.0,2,0.876564,0.581249,126.740740,4.778653,194.251120,14.585556,23.74305,37.98062,True,athens
266,266,104.281396,Entire home/apt,False,False,4.0,True,1,0,10.0,97.0,1,3.630743,0.115581,47.965227,1.807221,70.429904,1.051796,23.72669,38.00753,False,athens
4431,4431,1086.102684,Entire home/apt,False,False,6.0,False,0,0,8.0,93.0,3,6.156406,0.557863,201.149744,13.982142,462.250741,8.276985,-0.20862,51.48600,True,london
694,694,104.281396,Entire home/apt,False,False,2.0,True,0,0,10.0,99.0,0,1.851616,0.355356,78.934748,2.976168,122.312249,9.183948,23.75631,37.97754,True,athens
3261,3261,214.021967,Private room,False,True,2.0,False,1,0,10.0,100.0,1,3.772125,0.723740,297.789128,20.699653,675.281295,12.091475,-0.08895,51.53221,True,london
1199,1199,202.480974,Private room,False,True,2.0,True,1,0,10.0,98.0,1,4.385748,0.428070,172.371740,6.655472,338.083576,7.426560,2.21361,41.40911,False,barcelona
927,927,376.584726,Entire home/apt,False,False,6.0,True,0,1,10.0,100.0,2,0.693693,0.663922,427.295429,16.099525,651.795663,9.733877,23.73000,37.97100,False,athens
348,348,539.685515,Entire home/apt,False,False,6.0,True,0,1,10.0,93.0,3,5.309254,2.983305,75.550138,4.000430,94.482251,6.583659,4.87100,52.41900,True,amsterdam


In [42]:
#Removing unnecessary colunm 'Unamed:0'
cleaned_dataset = one_dataframe.drop(columns =  ['Unnamed: 0'])

In [43]:
# Categories
cleaned_dataset['room_type'] = cleaned_dataset['room_type'].astype('category')
cleaned_dataset['location'] = cleaned_dataset['location'].astype('category')

In [44]:
# Person Capacity
cleaned_dataset['person_capacity'] = cleaned_dataset['person_capacity'].astype('int')

In [45]:
# Using dtype_diet package to check memory usage and better data types
report_on_dataframe(cleaned_dataset)

,Current dtype,Proposed dtype,Current Memory (MB),Proposed Memory (MB),Ram Usage Improvement (MB),Ram Usage Improvement (%)
Column,,,,,,
realSum,float64,None,403.960938,NaN,NaN,NaN
room_type,category,None,227.382812,NaN,NaN,NaN
room_shared,bool,None,227.228027,NaN,NaN,NaN
room_private,bool,None,227.228027,NaN,NaN,NaN
person_capacity,int64,int8,403.960938,227.228027,176.732910,43.75
host_is_superhost,bool,None,227.228027,NaN,NaN,NaN
multi,int64,int8,403.960938,227.228027,176.732910,43.75
biz,int64,int8,403.960938,227.228027,176.732910,43.75
cleanliness_rating,float64,float16,403.960938,252.475586,151.485352,37.50


In [46]:
# Implementing the types that were suggested
report = report_on_dataframe(cleaned_dataset)
cleaned_dataset = optimize_dtypes(cleaned_dataset, report)

In [47]:
# Comparing the original dataset with the optimised one
print('=' * 20, 'ORIGINAL DATASET', '=' * 20)
print()
one_dataframe.info()
print()
print('=' * 20, 'OPTIMISED DATASET', '=' * 20)
print()
cleaned_dataset.info()

==================== ORIGINAL DATASET ====================

<class 'pandas.core.frame.DataFrame'>
Int64Index: 51707 entries, 0 to 4613
Data columns (total 22 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   Unnamed: 0                  51707 non-null  int64  
 1   realSum                     51707 non-null  float64
 2   room_type                   51707 non-null  object 
 3   room_shared                 51707 non-null  bool   
 4   room_private                51707 non-null  bool   
 5   person_capacity             51707 non-null  float64
 6   host_is_superhost           51707 non-null  bool   
 7   multi                       51707 non-null  int64  
 8   biz                         51707 non-null  int64  
 9   cleanliness_rating          51707 non-null  float64
 10  guest_satisfaction_overall  51707 non-null  float64
 11  bedrooms                    51707 non-null  int64  
 12  dist                        5

In [48]:
# Taking a last look on the dataframe before loading it
cleaned_dataset.head()

,realSum,room_type,room_shared,room_private,person_capacity,host_is_superhost,multi,biz,cleanliness_rating,guest_satisfaction_overall,bedrooms,dist,metro_dist,attr_index,attr_index_norm,rest_index,rest_index_norm,lng,lat,is_weekday,location
0,536.396682,Entire home/apt,False,False,5,False,0,1,9.0,89.0,1,1.351201,0.212346,390.776775,19.001549,1030.738507,47.550371,2.35900,48.86800,False,paris
1,290.101594,Private room,False,True,2,True,0,0,10.0,97.0,1,0.699821,0.193710,518.478270,25.211044,1218.658866,56.219575,2.35385,48.86282,False,paris
2,445.754497,Entire home/apt,False,False,4,False,0,1,10.0,100.0,1,0.968982,0.294343,432.689942,21.039580,1069.894793,49.356741,2.36023,48.86375,False,paris
3,211.343089,Private room,False,True,2,False,0,0,10.0,94.0,1,3.302319,0.234740,444.555284,21.616533,902.856370,41.650870,2.31714,48.87475,False,paris
4,266.334234,Entire home/apt,False,False,2,True,0,0,9.0,88.0,1,1.402430,0.055052,1013.458689,49.279502,1348.063511,62.189313,2.33408,48.85384,False,paris


## LOAD
Finally, I have a dataset to use in my future analysis. Please notice I haven't interfered radically with the data itself. For example, I have not checked outliers, cleaned missing data or any thing like that. That is because these will probably be done in each individual project when necessary so that I don't bias my conclusions or machine learning models.

In [49]:
cleaned_dataset.to_csv('/kaggle/working/airbnb_prices_merged.csv')

## FINAL REMARKS
As I am a beginner in this 'data science world', I am more than happy to receive some comments and suggestions on this or any other project. My email for contact is rocha.felipeferreira@gmail.com.